# Script to read in 2 data sets and compare them

Note: will get nonsense unless the two have the same number of galaxies

In [ ]:
# Imports
import h5py
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rc('font',size=20)

In [ ]:
# Parameters
outputDir='../../../output/'
dataSet1='cooling_test_old'
dataSet2='cooling_test_new'
coolingDiagnostics=True
filePrefix='SA_output_'
filePostfix='.h5'
Hubble_h=0.673
snap='58'
firstFile=40
lastFile=40
property='XrayLum'
logplot=False
#property='StellarMass'
#logplot=True
#property='HotGas'
#logplot=True
#property='CoolingRate'
#logplot=True
property='CoolingTest'
logplot=True
central=True

In [ ]:
# Read in first data set
dataDir=outputDir+dataSet1+'/'
# First determine the size of the arrays that we need to hold the data
nGal1=0
for iFile in range(firstFile,lastFile+1):
    # The following line closes the file at the end of the loop
    with h5py.File(dataDir+filePrefix+'%i'%iFile+filePostfix,'r') as f:
        nGal1+=len(f[snap])

# Declare numpy arrays to hold the data
Prop1=np.empty(nGal1)
Mvir1=np.empty(nGal1)
if central: Type1=np.empty(nGal1)

# Now read in the data
iGal=0
for iFile in range(firstFile,lastFile+1):
    # The following line closes the file at the end of the loop
    with h5py.File(dataDir+filePrefix+'%i'%iFile+filePostfix,'r') as f:
        nGalFile=len(f[snap])
        Prop1[iGal:iGal+nGalFile]=f[snap][property]
        Mvir1[iGal:iGal+nGalFile]=f[snap]['Mvir']
        Type1[iGal:iGal+nGalFile]=f[snap]['Type']
        iGal+=nGalFile
        
# Keep only centrals, if requested
if central:
    Prop1=Prop1[np.where(Type1==0)]
    Mvir1=Mvir1[np.where(Type1==0)]
    nGal1=Prop1.size
print('nGal1=',nGal1)
        
# Read in second data set
dataDir=outputDir+dataSet2+'/'
# First determine the size of the arrays that we need to hold the data
nGal2=0
for iFile in range(firstFile,lastFile+1):
    # The following line closes the file at the end of the loop
    with h5py.File(dataDir+filePrefix+'%i'%iFile+filePostfix,'r') as f:
        nGal2+=len(f[snap])

# Declare numpy arrays to hold the data
Prop2=np.empty(nGal2)
Mvir2=np.empty(nGal2)
if coolingDiagnostics:
    dt_ratio=np.empty(nGal2)
    tau_ratio=np.empty(nGal2)
if central: Type2=np.empty(nGal2)

# Now read in the data
iGal=0
for iFile in range(firstFile,lastFile+1):
    # The following line closes the file at the end of the loop
    with h5py.File(dataDir+filePrefix+'%i'%iFile+filePostfix,'r') as f:
        nGalFile=len(f[snap])
        Prop2[iGal:iGal+nGalFile]=f[snap][property]
        Mvir2[iGal:iGal+nGalFile]=f[snap]['Mvir']
        Type2[iGal:iGal+nGalFile]=f[snap]['Type']
        if coolingDiagnostics:
            dt_ratio[iGal:iGal+nGalFile]=f[snap]['dt_ratio']
            tau_ratio[iGal:iGal+nGalFile]=f[snap]['tau_ratio']
        iGal+=nGalFile
        
# Keep only centrals, if requested
if central:
    index=np.where(Type2==0)
    Prop2=Prop2[index]
    Mvir2=Mvir2[index]
    if coolingDiagnostics:
        dt_ratio=dt_ratio[index]
        tau_ratio=tau_ratio[index]
    nGal2=Prop2.size
print('nGal2=',nGal2)

In [ ]:
if nGal1 != nGal2:
    print('Warning nGal1 != nGal2, so only some of the following galaxies match up')
    nGal=min(nGal1,nGal2)
    nGal=1000
    Prop1=Prop1[:nGal]
    Prop2=Prop2[:nGal]

In [ ]:
if coolingDiagnostics:

    Colour=np.where(tau_ratio<=1,'r','g')
    Colour[np.logical_and(tau_ratio>1,dt_ratio>=np.log(tau_ratio))]='b'
    Y=10.
    tau_ratio_norm=Y**3*3.*np.pi/(20.*(Y-np.arctan(Y))**2)

# Plot the two properties agains one another (only really makes sense if galaxy numbers are equal)
plt.figure(0,figsize=(18,6))
plt.subplot(1,2,1)
if logplot:
    if coolingDiagnostics:
        index=np.where(Colour=='r')
        plt.loglog(Prop1[index],Prop2[index],'.r',label='tau_ratio<=1')
        index=np.where(Colour=='b')
        plt.loglog(Prop1[index],Prop2[index],'.b',label='tau_ratio>1 & dt_ratio>=log(tau_ratio)')
        index=np.where(Colour=='g')
        plt.loglog(Prop1[index],Prop2[index],'.g',label='tau_ratio>1 & dt_ratio<log(tau_ratio)')
    else:
        plt.loglog(Prop1,Prop2,'.')
else:
    if coolingDiagnostics:
        index=np.where(Colour=='r')
        plt.plot(Prop1[index],Prop2[index],'.r',label='tau_ratio<=1')
        index=np.where(Colour=='g')
        plt.plot(Prop1[index],Prop2[index],'.g',label='tau_ratio>1 & dt_ratio<log(tau_ratio)')
        index=np.where(Colour=='b')
        plt.plot(Prop1[index],Prop2[index],'.b',label='tau_ratio>1 & dt_ratio>=log(tau_ratio)')
    else:
        plt.plot(Prop1,Prop2,'.')
plt.plot(Prop1,Prop1,'k-')
plt.plot(Prop2,Prop2,'k-')
plt.xlabel(dataSet1)
plt.ylabel(dataSet2)
plt.title(property)
plt.subplot(1,2,2)
if logplot:
    if coolingDiagnostics:
        index=np.where(Colour=='r')
        plt.semilogx(Prop1[index],np.log10(Prop2[index]/Prop1[index]),'.r',label='tau_ratio<=1')
        plt.semilogx(Prop1[index],np.log10(np.sqrt(tau_ratio_norm*tau_ratio[index])),'k:')
        plt.semilogx(Prop1[index],np.log10(np.sqrt(tau_ratio_norm*Hubble_h*tau_ratio[index])),'k--')
        index=np.where(Colour=='b')
        plt.semilogx(Prop1[index],np.log10(Prop2[index]/Prop1[index]),'.b',label='tau_ratio>1 & dt_ratio>=log(tau_ratio)')
        index=np.where(Colour=='g')
        plt.semilogx(Prop1[index],np.log10(Prop2[index]/Prop1[index]),'.g',label='tau_ratio>1 & dt_ratio<log(tau_ratio)')
    else:
        plt.semilogx(Prop1,np.log10(Prop2/Prop1),'.')
    plt.ylabel(r'$\log_{10}($'+dataSet2+'/'+dataSet1+'$)$')
else:
    if coolingDiagnostics:
        index=np.where(Colour=='r')
        plt.plot(Prop1[index],Prop2[index]-Prop1[index],'.r',label='tau_ratio<=1')
        index=np.where(Colour=='g')
        plt.plot(Prop1[index],Prop2[index]-Prop1[index],'.g',label='tau_ratio>1 & dt_ratio<log(tau_ratio)')
        index=np.where(Colour=='b')
        plt.plot(Prop1[index],Prop2[index]-Prop1[index],'.b',label='tau_ratio>1 & dt_ratio>=log(tau_ratio)')
    else:
        plt.plot(Prop1,Prop2-Prop1,'.')
    plt.ylabel(dataSet2+'-'+dataSet1)
plt.plot(Prop1,0.*Prop1,'k-')
#plt.plot([max(Prop1),],[np.log10(dt_ratio[0]/np.sqrt(tau_ratio_norm)/Hubble_h),],'ko',ms=10)
#plt.plot([max(Prop1),],[np.log10(dt_ratio[0]/np.sqrt(tau_ratio_norm)),],'ko',ms=10)
#plt.plot([max(Prop1),],[np.log10(1-np.exp(-dt_ratio[0]))-np.log10(dt_ratio[0]),],'ko',ms=10)#plt.plot([1,],[0,],'ko',ms=10)
#plt.plot([1,],[np.log10(1-1./(1+tau_ratio_norm/dt_ratio[0])),],'ko',ms=10)
#plt.plot([1,],[np.log10(1-np.exp(-dt_ratio[0])),],'ko',ms=10)
#red
#plt.plot([dt_ratio[0]*np.sqrt(max(tau_ratio/tau_ratio_norm)),],[np.log10(1-1./(1+dt_ratio[0]*max(tau_ratio)))-np.log10(dt_ratio[0]*np.sqrt(max(tau_ratio/tau_ratio_norm))),],'ko',ms=10)
#blue
#plt.plot([dt_ratio[0]*np.sqrt(max(tau_ratio/tau_ratio_norm)),],[np.log10(1-1./(max(tau_ratio*tau_ratio_norm)*(1+dt_ratio[0]-np.log(tau_ratio_norm))))-np.log10(dt_ratio[0]*np.sqrt(max(tau_ratio/tau_ratio_norm))),],'ko',ms=10)
plt.title(property)
#plt.legend(loc=5)
plt.legend()
plt.savefig('figs/'+property+'.png',bbox_inches='tight')

In [ ]:
plt.loglog(dt_ratio,tau_ratio,'.',)
plt.xlabel('dt_ratio')
plt.ylabel('tau_ratio')

In [ ]:
1-1./(1+dt_ratio[0]*max(tau_ratio))

In [ ]:
1+dt_ratio[0]-np.log(tau_ratio_norm)